In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.set_option('display.max_columns', None)

In [ ]:
test_file = '/kaggle/input/playground-series-s3e22/test.csv'
train_file = '/kaggle/input/playground-series-s3e22/train.csv'

In [ ]:
test_data = pd.read_csv(test_file)
test_data.head()

In [ ]:
train_data = pd.read_csv(train_file)
train_data.head()

In [ ]:
# from sklearn.tree import DecisionTreeClassifier -- This needs categorical data changed to numerical

# model = DecisionTreeClassifier()
# model.fit(X_train,y_train)

In [ ]:
##Adding missing data
##EA was done as part of another notebook.  Will try and keep the rest of the notebooks clean.
for column in train_data.select_dtypes(include=["object"]):
    train_data[column]=train_data[column].fillna("no_measure_recorded")
    
for column in test_data.select_dtypes(include=["object"]):
    test_data[column]=test_data[column].fillna("no_measure_recorded")
    
for column in train_data.select_dtypes(include=["int64"]):
    train_data[column]=train_data[column].fillna(0)
    
for column in test_data.select_dtypes(include=["int64"]):
    test_data[column]=test_data[column].fillna(0)

for column in train_data.select_dtypes(include=["float64"]):
    train_data[column]=train_data[column].fillna(0)
    
for column in test_data.select_dtypes(include=["float64"]):
    test_data[column]=test_data[column].fillna(0)

In [ ]:
X_train = train_data.drop(columns=["outcome","id","hospital_number"])
y_train = train_data["outcome"]
X_test = test_data

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

categorical_cols = [cname for cname in X_train.columns if
                   X_train[cname].dtype == object]

categorical_cols

In [ ]:
categorical_cols_indices = [X_train.columns.get_loc(cname) for cname in categorical_cols]

In [ ]:
## ValueError: Found unknown categories ['moderate'] in column 6 during transform -- data in the test wasn't present in the training
## addedhandle_unknown='ignore' to OneHotEncoder
##after adding ignore  ValueError: Specifying the columns using strings is only supported for pandas DataFrames
## Got the indicies of the cat columns
##Problem might have been the need to restart kernal.  Will make a habit to restart after every error
preprocessor = ColumnTransformer(
transformers=[
    ('cat',OneHotEncoder(drop='first', handle_unknown='ignore'),categorical_cols_indices)
],remainder='passthrough')

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# /opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
#   warnings.warn(

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
test_data["outcome"] = y_pred

In [ ]:
output = test_data[['id', 'outcome']]
output.to_csv('outputII.csv', index=False)
#Score: 0.74390  Rank upper 400's  Not Too Shabby for my first try and brute force method.
#ScoreII: 0.76829